In [136]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../data/processed/replication_test_2023_09_24_11_55_06/with_factors.csv")

In [6]:
df.columns

Index(['Unnamed: 0', 'sequence_dna', 'fitness', 'p_value', 'standard_error',
       'mutated_wildtype_dna', 'mutated_amino_acids', 'minimum_free_energy'],
      dtype='object')

In [139]:
variables = ["minimum_free_energy", "minimum_free_energy"]
group_by = ["mutated_amino_acids"]
significant_threshold = 0.05

In [140]:
grouped_df = df.groupby(["mutated_amino_acids"])
grouped_dict = {key: group for key, group in grouped_df}

In [141]:
grouped_correlations = {}

for key, group_df in grouped_dict.items():
    correlation_matrix, p_value = stats.spearmanr(
        group_df[["fitness"]],
        group_df[variables],
        axis=0,
    )

    if isinstance(correlation_matrix, np.ndarray):
        grouped_correlations[key] = np.concatenate(
            [correlation_matrix[0][1:], p_value[0][1:]]
        )

    if isinstance(correlation_matrix, float):
        grouped_correlations[key] = np.array(
            [correlation_matrix, p_value]
        )


/Users/tobiaochsner/Documents/ETH/GeneticFitness/.venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/Users/tobiaochsner/Documents/ETH/GeneticFitness/.venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/Users/tobiaochsner/Documents/ETH/GeneticFitness/.venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/Users/tobiaochsner/Documents/ETH/GeneticFitness/.venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  

In [142]:
correlation_df = pd.DataFrame.from_dict(
        grouped_correlations,
        orient="index",
        columns=[f"{variable}_correlation" for variable in variables]
        + [f"{variable}_correlation_p_value" for variable in variables],
    )

In [143]:
for variable in variables:
    correlation_df[f"{variable}_significant"] = correlation_df[
        f"{variable}_correlation_p_value"
    ] < significant_threshold

ValueError: Cannot set a DataFrame with multiple columns to the single column minimum_free_energy_significant

In [144]:
correlation_df

,minimum_free_energy_correlation,minimum_free_energy_correlation,minimum_free_energy_correlation_p_value,minimum_free_energy_correlation_p_value
"(*A*,)",NaN,NaN,NaN,NaN
"(*AI,)",NaN,NaN,NaN,NaN
"(*D*,)",NaN,NaN,NaN,NaN
"(*DA,)",-0.178571,-0.178571,0.701658,0.701658
"(*DC,)",1.000000,1.000000,0.000000,0.000000
...,...,...,...,...
"(YTH,)",NaN,NaN,NaN,NaN
"(YTL,)",NaN,NaN,NaN,NaN
"(YTT,)",NaN,NaN,NaN,NaN
"(YV*,)",NaN,NaN,NaN,NaN


In [145]:
for variable in variables:
    sns.histplot(correlation_df, x=f"{variable}_correlation", hue=f"{variable}_significant")
    plt.show()

ValueError: Could not interpret value `minimum_free_energy_significant` for parameter `hue`